In [1]:
from nlp_models.exaone.load_model import get_exaone

lm_cache_dir = "/data1/bumjin/datahub"
model, tokenizer = get_exaone('exaone', '8b', lm_cache_dir, 'cpu')
print(model)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ExaoneForCausalLM(
  (transformer): ExaoneModel(
    (wte): Embedding(102400, 4096, padding_idx=0)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-31): 32 x ExaoneBlock(
        (ln_1): ExaoneRMSNorm()
        (attn): ExaoneAttention(
          (attention): ExaoneSdpaAttention(
            (rotary): ExaoneRotaryEmbedding()
            (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
          )
        )
        (ln_2): ExaoneRMSNorm()
        (mlp): ExaoneGatedMLP(
          (c_fc_0): Linear(in_features=4096, out_features=14336, bias=False)
          (c_fc_1): Linear(in_features=4096, out_features=14336, bias=False)
          (c_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act): SiLU

In [2]:
model.config

ExaoneConfig {
  "_name_or_path": "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
  "activation_function": "silu",
  "architectures": [
    "ExaoneForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct--configuration_exaone.ExaoneConfig",
    "AutoModelForCausalLM": "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct--modeling_exaone.ExaoneForCausalLM",
    "AutoModelForSequenceClassification": "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct--modeling_exaone.ExaoneForSequenceClassification"
  },
  "bos_token_id": 1,
  "embed_dropout": 0.0,
  "eos_token_id": 361,
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "layer_norm_epsilon": 1e-05,
  "max_position_embeddings": 4096,
  "model_type": "exaone",
  "num_attention_heads": 32,
  "num_key_value_heads": 8,
  "num_layers": 32,
  "pad_token_id": 0,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_v

In [3]:
from nlp_models.utils.find_vocab_index import find_vocab_index, find_inclusion_vocab_index, decode_token_index

find_vocab_index(tokenizer, 'yes'), find_inclusion_vocab_index(tokenizer, 'yes')

print("====================")
for index in find_vocab_index(tokenizer, 'yes'):
    if index is not None:
        print(index, decode_token_index(tokenizer, index))
print("====================")
for index in find_inclusion_vocab_index(tokenizer, 'yes'):
    print(index, decode_token_index(tokenizer, index))

64173 yes
17050 Yes
17050 Yes
5863 Ġeyes
38501 ĠYES
72405 eyes
74897 Yesterday
18509 Ġyesterday
64376 ĠBayes
49495 Ġdyes
34687 ĠBayesian
52217 ĠEyes
50711 ĠHayes
52847 Ġpolyester
8985 Ġyes
72984 ĠReyes
64173 yes
69035 ĠYesterday
8277 ĠYes


In [4]:
text = "I ate bread because I was feeling blue. What do you think?"
inputs = tokenizer(text, return_tensors="pt")
output = model.generate(inputs['input_ids'].cuda(), max_new_tokens=25)
output = output[0].tolist()

print(text)
print(tokenizer.decode(output[inputs['input_ids'].shape[-1]:], 
                       skip_special_tokens=True, 
                       pad_token_id=tokenizer.eos_token_id))

In [5]:
messages = [
    {"role": "system", 
     "content": "You are EXAONE model from LG AI Research, a helpful."},
    {"role": "user", "content": "ttompt"}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

tokenizer.decode(input_ids[0], skip_special_tokens=True)

'[|system|]You are EXAONE model from LG AI Research, a helpful.\n[|user|]ttompt\n[|assistant|]'